In [ ]:
def calculate_course_handicap(handicap_index, course_rating, slope_rating, par=None):
    """
    Calculate the course handicap based on the golfer's handicap index, 
    course rating, slope rating, and optional par.

    Args:
        handicap_index (float): The golfer's handicap index.
        course_rating (float): The course rating.
        slope_rating (int): The slope rating of the course.
        par (int, optional): Par for the course (used in full formula).

    Returns:
        int: The calculated course handicap (rounded to nearest integer).
    """
    if par is not None:
        # Full USGA formula
        course_handicap = handicap_index * (slope_rating / 113) + (course_rating - par)
    else:
        # Simplified formula
        course_handicap = handicap_index * (slope_rating / 113)

    return round(course_handicap)

In [ ]:
def remove_duplicate_name_date_rows(parsed_rows):
    """
    Removes rows that have the same golfer name and date played.
    Keeps only the first occurrence.
    """
    seen = set()
    filtered = []

    for row in parsed_rows:
        golfer_name = row[1]
        date_played = row[23]
        total = row[22]
        key = (golfer_name, date_played, total)

        if key not in seen:
            seen.add(key)
            filtered.append(row)

    return filtered

In [ ]:
# Parse GolfData spreadsheet in notebook cell
import pandas as pd
from pathlib import Path
from datetime import datetime

# Path to your file
file_path = Path("/home/justin/JustInternetAI/Calcutta/Data/DirtyData/hole2024.xlsx")  # Replace if needed

# Load the raw Excel data
df_raw = pd.read_excel(file_path, header=None)

event = "individual"

# Columns to extract in output
output_columns = [
    "Event", "Player", "Handicap", "Tee"
] + [str(i) for i in range(1, 19)] + ["Total"] + ["Date"] + ["Index"] + ["CR", "SR"]

# Placeholder for cleaned rows
parsed_rows = []
current_name = None
current_tee = None

# Iterate over rows
for idx in range(len(df_raw)):
    if idx <= 0:
        continue  # Skip header rows
    row = df_raw.iloc[idx]
    name = row[1]
    
    # If there's a new golfer name in this row, update context only
    if pd.notnull(name) and isinstance(name, str):
        current_name = name.strip()
        current_tee = row[6]
        if pd.notnull(row[8]) and pd.notnull(row[9]):
            cr = float(row[8])
            sr = float(row[9])
        else:
            print(f"⚠️ Missing course rating or slope rating in row {idx + 2}")
            continue  # Skip this row; next row has valid data
        continue

    # If no current name is set, skip (invalid row before name row)
    if not current_name:
        continue

    # Extract relevant columns from this row
    try:
        date_played = pd.to_datetime(row[2]).date() if pd.notnull(row[2]) else None
        if pd.notnull(row[3]):
            handicap_index = float(row[3])
        course_handicap = calculate_course_handicap(handicap_index,cr,sr)
        hole_scores = row[10:19].tolist() + row[20:29].tolist() # Holes 1–18
        total = row[30]

        parsed_rows.append([
            event,
            current_name,
            course_handicap,
            current_tee
        ] + hole_scores + [total] + [date_played] + [handicap_index] + [cr, sr])

    except Exception as e:
        print(f"⚠️ Skipping row {idx+1}:  handicap_index ={handicap_index}, cr = {cr}, sr = {sr}    {e}")


parsed_rows = remove_duplicate_name_date_rows(parsed_rows)

# Create cleaned DataFrame
df_cleaned = pd.DataFrame(parsed_rows, columns=output_columns)

# Save to Excel
output_path = Path("/home/justin/JustInternetAI/Calcutta/Data/DirtyData/cleaned_hole_by_hole_2024.xlsx")
df_cleaned.to_excel(output_path, index=False)

print(f"✅ Parsed and saved to: {output_path}")

In [ ]:
# Using simplified version:
hcp = calculate_course_handicap(handicap_index=14.4, course_rating=70.5, slope_rating=128)
print(f"Course Handicap: {hcp}")
